# Trash Object Detection Using YOLOv8

#### Problem Statement 
Public trash inflicts severe consequences on the environment, quality of life, and long-term sustainability (EPA, 2020). As a result, this project proposes to address these challenges by building a model to deploy on a robot to identfy and collect trash.

#### Purpose
This notebook is contains the components to deploy an trash detection model using YOLOv5 and YOLOv8. The notebook includes the following: (1) Installation and Setup of YOLOv8, (2) Building and Training YOLOv8x Model on the TACO dataset, (3) YOLOv8x Model Evaluation, and (4) Trash Detection Model Deployment on Freenove Tank Robot: (a) implement real-time trash detection using the robot's camera and the trained model and (b) program the robot to move towards and pick-up trash if detected or keep searching for trash if no trash was detected. 

## Installation and Preparation

### YOLOv8
For information on installation and preperation, see https://github.com/mattiegisselbeck/trash-detection.

The cell below are a list of CLI commands that clone the 'trash-detection' and 'yolov5' repositories, sets up a Python 3.8 Anaconda environment named 'yolov8', and installs the Ultralytics package for YOLO model development. The data from the 'trash-detection' repository should be moved to yolov5/datasets.

### Build and Train YOLOv8x Model

The cell below imports the YOLO class from the 'ultralytics' library. Then, creates a new model instance using the 'yolov8x.yaml' configuration file and trains the model on a dataset specified in 'data.yaml' for three epochs.

In [ ]:
from ultralytics import YOLO

# Build Trash Detection Model
model = YOLO("yolov8x.yaml") 

# Train Trash Detection YOLOv8
results = model.train(data="./data.yaml", epochs=3)

### Test YOLOv8x Model


The code cell below loads a pre-trained YOLOv8x model for trash detection and then using it to perform object detection on a test image. The results of the detection, such as the identified objects and their locations in the image, are stored in the 'results' variable.

In [3]:
model = YOLO("./models/trash-detection-yolov8x.pt") # Trash Detection YOLOv8x Model

# Test Trash Detection YOLOv8x Model
results = model("./test/test0.JPG")


image 1/1 /Users/mattiegisselbeck/Documents/GitHub/trash-detection/test/test0.JPG: 640x480 2 Cans, 2 Cartons, 4645.2ms
Speed: 2.4ms preprocess, 4645.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)


### YOLOv8 on Freenove Robot / Raspberry Pi (Draft)

The code cell(s) below are written for YOLOv8x deployment on the Freenove robot to identify trash (using the robot's camera), using the model and sensors to calculate the distance the robot needs to move in order to pick up the trash, picking up the trash, and if no trash is detected to move forward six inches. These commands will be formatted into a loop for continous trash detection.

In [ ]:
import os
import time
import pigpio
from camera import Camera
from motor import Motor
from infrared import Infrared
import RPi.GPIO as GPIO
import time
from Motor import *
from servo import *

class Detect:
    def __init__(self):
        # Initialize pigpio
        os.system("sudo pigpiod")
        time.sleep(1)  # Wait for pigpiod to initialize

        # Initialize components
        self.camera = Camera()
        self.motor = Motor()
        self.infrared = Infrared()

    def detect_trash(self):
        while True:
            trash_detected, distance = self.camera.look_for_trash()

            if not trash_detected:
                self.motor.move_forward(6)  # Move forward 6 inches
            else:
                self.motor.move_distance(distance)  # Move towards the trash
                self.motor.pick_up_trash()
                self.motor.drop_trash()

                # After handling the trash, loop back to start

In [ ]:
def find_trash():
    picam2 = Picamera2()
    preview_config = picam2.create_preview_configuration(main={"size": (640, 480)},transform=Transform(hflip=1,vflip=1))
    picam2.configure(preview_config)
    picam2.start_preview(Preview.QTGL)
    picam2.start()
    time.sleep(2)
    metadata = picam2.capture_file("image.jpg")
    print(metadata)
    picam2.close()
    
    # Detect 
    model = YOLO("/Users/mattiegisselbeck/Documents/GitHub/trashbot/runs/detect/train4/weights/best.pt") 
    results = model("/path/to/image.jpg")

In [ ]:
class Move:
    def __init__(self):        
        GPIO.setwarnings(False)        
        self.trigger_pin = 27
        self.echo_pin = 22
        self.MAX_DISTANCE = 300               # define the maximum measuring distance, unit: cm
        self.timeOut = self.MAX_DISTANCE*60   # calculate timeout according to the maximum measuring distance
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.trigger_pin,GPIO.OUT)
        GPIO.setup(self.echo_pin,GPIO.IN)
        
    def pulseIn(self,pin,level,timeOut): # obtain pulse time of a pin under timeOut
        t0 = time.time()
        while(GPIO.input(pin) != level):
            if((time.time() - t0) > timeOut*0.000001):
                return 0;
        t0 = time.time()
        while(GPIO.input(pin) == level):
            if((time.time() - t0) > timeOut*0.000001):
                return 0;
        pulseTime = (time.time() - t0)*1000000
        return pulseTime
    
    def get_distance(self):     # get the measurement results of ultrasonic module,with unit: cm
        distance_cm2=[0.0,0.0,0.0,0.0,0.0]
        for i in range(5):
            GPIO.output(self.trigger_pin,GPIO.HIGH)      # make trigger_pin output 10us HIGH level 
            time.sleep(0.00001)     # 10us
            GPIO.output(self.trigger_pin,GPIO.LOW) # make trigger_pin output LOW level 
            pingTime = self.pulseIn(self.echo_pin,GPIO.HIGH,self.timeOut)   # read plus time of echo_pin
            distance_cm2[i] = pingTime * 340.0 / 2.0 / 10000.0     # calculate distance with sound speed 340m/s
        distance_cm2=sorted(distance_cm2)
        return  distance_cm2[2]
    
    def run_motor(self,distance):
        if(distance!=0):
            if distance < 45 :
                self.PWM.setMotorModel(-1500,-1500) #Back
                time.sleep(0.4)
                self.PWM.setMotorModel(-1500,1500)  #Left
                time.sleep(0.2)         
            else :
                self.PWM.setMotorModel(1500,1500)   #Forward
            
    def run(self):
        self.PWM=Motor()
        while True:
            distance = self.get_distance()
            time.sleep(0.2)
            #print ("The distance is "+str(distance)+"CM")
            self.run_motor(distance)

ultrasonic=Ultrasonic()         
if __name__ == '__main__':     # Program entrance
    print ('Program is starting...')
    servo=Servo()
    servo.setServoPwm('0',90)
    servo.setServoPwm('1',140)
    try:
        ultrasonic.run()
    except KeyboardInterrupt:  # When 'Ctrl+C' is pressed, the child program destroy() will be  executed.
        PWM.setMotorModel(0,0)
        servo.setServoPwm('0',90)
        servo.setServoPwm('1',140)
        print ("\nEnd of program")